In [8]:
from Income import Income
from compass import Compass
from Dice import Dice

import pandas as pd
import numpy as np
import torch
from IPython.display import clear_output

In [2]:
from Income import Income
d = Income()
d.target_torch_train.size()

torch.Size([24420])

## Load model, data structure and the cf generator

In [2]:
# m = torch.load("classifier_model.pt")
m = torch.load("classifier_model_compass.pt")
m.eval()

# d = Income()
d = Compass()
exp = Dice(d, m)

## Generate 5 cf's for every instance in the train set - INCOME

In [ ]:
n_instances = len(d.data_torch_train)
n_cfs = 5
cfs = torch.zeros((d.data_torch_train.shape[0], n_cfs, d.data_torch_train.shape[1]))

for n_instance in np.arange(n_instances):
    clear_output(wait=True)
    print('instance ' + str(n_instance+1) + '/' + str(n_instances))
    print('percentage: ' + str(round(n_instance*100/n_instances, 2)) + '%')
    x = d.data_torch_train[n_instance]
    cfs[n_instance] = exp.generate_cfs(x, total_cfs=n_cfs)
    if n_instance % 100 == 0:
        torch.save(cfs, 'cfs.pt')

torch.save(cfs, 'cfs.pt')
print('Done!')

## Generate 5 cf's for every instance in the train set - COMPAS

In [12]:
n_instances = len(d.data_torch_train)
n_cfs = 5
instances = np.zeros((n_instances, 6)).astype(object)
cfs = np.zeros((n_instances*n_cfs, 6)).astype(object)
    
for n_instance in np.arange(n_instances):
    clear_output(wait=True)
    print('instance ' + str(n_instance+1) + '/' + str(n_instances))
    print('percentage: ' + str(round(n_instance*100/n_instances, 2)) + '%')
    
    x = d.data_torch_train[n_instance]
    output = exp.generate_cfs(x, total_cfs=n_cfs, output='df', diversity_weight=50)
    if output == 'sorry':
        continue
    
    cfs_i = output[0]
    cfs_i['age'] = cfs_i['age'].astype('float').round().astype('int')
    cfs_i['priors_count'] = cfs_i['priors_count'].astype('float').round().astype('int')
    cfs_i['recidivism'] = cfs_i['recidivism'].astype('bool')
    cfs_i = cfs_i.to_numpy()
    cfs[n_instance*5:n_instance*5+5,:] = cfs_i
    
    x_df = d.data_df_train.iloc[n_instance].copy()
    target = round(m(x).item())
    x_df['recidivism'] = bool(target)
    instances[n_instance] = x_df.to_numpy()
    
    if n_instance % 10 == 0:
        cfs_df = pd.DataFrame(cfs[0:n_instance*5+5,:])
        instances_df = pd.DataFrame(instances[0:n_instance+1,:])
        
        cfs_df.to_csv('cfs.csv', index=False, header=False)
        instances_df.to_csv('instances.csv', index=False, header=False)
    


instance 1482/5410
percentage: 27.38%


KeyboardInterrupt: 

In [ ]:
cfs[0:50,:]

In [6]:
instances[0:10]

array([['Felony', 'African-American', 'Male', 34, 6, True],
       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]], dtype=object)